# Signal

> Signal processing utilities that make complete sense

In [ ]:
#| default_exp signal

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from scipy import signal
import torch, numpy as np, torch.nn.functional as F

In [ ]:
#| export
def butterworth(waveform_array, freq_range, btype, fs=125, order=2): # Recommend playing around with the order as well
    # Butterworth filter
    sos = signal.butter(order, freq_range, fs=fs, btype=btype, output='sos')
    filtered = signal.sosfilt(sos, waveform_array)
    return filtered

In [ ]:
#| export
def stft(signal_array,
        n_fft=256, # number of ffts to perform
        win_length=(2*125), # window of ffts
        pad_mode='reflect',
        pad_win_length_to_nfft=True, # indicator to pad the end of the sequence with 0s for nfft-win_legnth to ensuure correct size output
        center=False,
        hop_length=(125), 
        normalized=True,
        decibel_scale=False,
        return_complex=True,
        onesided=True,
        channel_stft_means=None, # precalculated stft channel means
        channel_stft_stds=None # precalculated stft channel stds
        ):
    """
    in: [bs x n_vars x max_seq_len]
    out: [bs x n_vars x n_fft // 2 + 1 x stft_len]
    """
    if isinstance(signal_array, np.ndarray):
        signal_array = torch.from_numpy(signal_array)
    if pad_win_length_to_nfft:
        signal_array = F.pad(signal_array, (0, n_fft - win_length), pad_mode) # constant pad with 0 
    shape = signal_array.size()
    signal_array = signal_array.reshape(-1, shape[-1]) # pack batch, this will do nothing for signals with no batch dim
    window = torch.hamming_window(win_length, periodic=False).to(signal_array.device)
    stfts = torch.stft(signal_array,
                        n_fft=n_fft, 
                        center=center, 
                        pad_mode=pad_mode, 
                        win_length = win_length,
                        onesided=onesided,
                        normalized=normalized,
                        window=window,
                        hop_length=hop_length, 
                        return_complex=return_complex).abs() # get magnitude only
    stfts = stfts.reshape(shape[:-1] + stfts.shape[-2:]) # unpack batch

    if decibel_scale:
        #stfts = 2 * stfts / window.sum() # https://dsp.stackexchange.com/questions/49184/stft-amplitude-normalization-librosa-library - multiply the spectra by 2 in order to retrieve the energy from negative frequencies and divide it by the window size
        eps = np.finfo(np.float64).eps
        stfts = 20*torch.log10(stfts+eps) # convert to decibel scale, more normal distribution - scale by batch in model? 
    if channel_stft_means is not None and channel_stft_stds is not None:
        assert channel_stft_means.shape == stfts.shape and channel_stft_stds.shape == stfts.shape, f"The channel_means and channel_stds must be equal to the shape generated by the stft: {stfts.shape}"
        stfts = (stfts - channel_stft_means) / channel_stft_stds
    return stfts

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()